# Interest Rate Risk: Duration and Convexity

In [26]:
def calculate_bond_price(
    par_value: float,
    coupon: float,
    yield_to_maturity: float,
    num_periods: int,
    num_periods_per_year: int,
) -> float:
    periodic_yield = yield_to_maturity / num_periods_per_year

    price = 0
    for period in range(1, num_periods + 1):
        price += coupon / (1 + periodic_yield) ** period
    
    price += par_value / (1 + periodic_yield) ** num_periods 

    return price


## Duration
Duartion is one of the ways to measure the risk of a bond. The formula for calculating duration is:
$$
D = \frac{
    \sum_{t=1}^{n}{
        \frac{tC}{(1+y)^t}
    } 
    + \frac{nM}{(1+y)^n}
}{P}
$$

where:

* t = Time period
* C = Coupon payment
* y = Periodic yield
* n = Number of periods
* M = Maturity value
* P = Current price

The duration of a bond is the average number of years untill the present value of the bond's cash flows equals the amount paid for the bond. 

Duration is also known as Macaulay Duration, named after the economist Frederick Macaulay.

In [28]:
def calculate_duration(
    par_value: float,
    coupon: float,
    yield_to_maturity: float,
    num_periods: int,
    num_periods_per_year: int,
) -> float:
    periodic_yield = yield_to_maturity / num_periods_per_year
    price = calculate_bond_price(
        par_value, coupon, yield_to_maturity, num_periods, num_periods_per_year
    )

    duration = 0
    for t in range(1, num_periods + 1):
        duration += (t * coupon) / (1 + periodic_yield) ** t

    duration += (num_periods * par_value) / (1 + periodic_yield) ** num_periods
    duration /= price
    duration /= num_periods_per_year

    return duration

In [32]:
par_value = 1000
coupon = 30
yield_to_maturity = 0.06
periods = 6
num_periods_per_year = 2

price = calculate_bond_price(par_value, coupon, yield_to_maturity, periods, num_periods_per_year)
duration = calculate_duration(
    par_value, coupon, yield_to_maturity, periods, num_periods_per_year
)

print(f"Price: ${price:.2f}")
print(f"Duration: {duration:.2f} years")

Price: $1000.00
Duration: 2.79 years


In [31]:
par_value = 100
coupon = 5
yield_to_maturity = 0.06
periods = 6
num_periods_per_year = 2

price = calculate_bond_price(par_value, coupon, yield_to_maturity, periods, num_periods_per_year)
duration = calculate_duration(
    par_value, coupon, yield_to_maturity, periods, num_periods_per_year
)

print(f"Price: ${price:.2f}")
print(f"Duration: {duration:.3f} years")

Price: $110.83
Duration: 2.684 years


## Modified Duration

The Modified duration of a bond is defined as the percentage change in price, over the percentage change in yield, i.e.

$$
\text{ModD} = - \frac{\Delta P/P}{\Delta y}
$$

Modified duration is closely related to duration (hence the name), and is usually calculated using the formula:

$$
\text{ModD} = \frac{D}{{1 + \frac{y}{n}}}
$$

where n is the number of compounding periods per year

Modified duration is a local measure and can only be used to approximate the price for small changes in yield (say around 1% or 100bp).

To get the new price using modified duration we can use:
$ P_{\text{new}} = P_{\text{old}}[1 - \text{ModD} \Delta y] $ 

In [10]:
def calculate_modified_duration(
    duration: float,
    yield_to_maturity: float,
    num_periods_per_year: int
) -> float:
    modified_duration = duration / (1 + yield_to_maturity / num_periods_per_year)
    return modified_duration

In [ ]:
par_value = 1000
coupon = 100
yield_to_maturity = 0.05
periods = 3
num_periods_per_year = 1

price = calculate_bond_price(
    par_value, coupon, yield_to_maturity, periods, num_periods_per_year
)
print(f"Price: ${price:.2f}")

duration = calculate_duration(
    par_value, coupon, yield_to_maturity, periods, num_periods_per_year
)
print(f"Duration: {duration:.3f} years")

convexity = calculate_modified_duration(
    duration, yield_to_maturity, num_periods_per_year
)
print(f"Modified Duration: {convexity:.2f}")

new_price = price * (1 - convexity * 0.0015)
print(f"Price in case of a 0.15% increase in yield: ${new_price:.2f}")

new_price = price * (1 - convexity * -0.0015)
print(f"Price in case of a 0.15% decrease in yield: ${new_price:.2f}")

Price: $1136.16
Duration: 2.753 years
Modified Duration: 2.62
Price in case of a 0.15% increase in yield: $1131.69
Price in case of a 0.15% decrease in yield: $1140.63


### Computational calculation of Modified Duration
The general way to compute the Modified duration of a bond is to calculate the price for a small increase and decrease in yield, and use these values to compute the modified duration.
Mathematically this can be expressed as:

$$
\text{ModD} = - \frac{(P_{+dy} - P_{-dy}) / P}{2dy}
$$

where:
* $dy$ - the change in price (or blip), taken to be 1bp, 2bp, or 0.5bp (depending on required precision)
* $P_{\pm dy}$ - The price when the yield is increased/decreased by $dy$

In [54]:
def calculate_modified_duration_comp(
    par_value: float,
    coupon: float,
    yield_to_maturity: float,
    num_periods: int,
    num_periods_per_year: int,
    blip: float = 0.00005,
) -> float:
    price = calculate_bond_price(
        par_value, coupon, yield_to_maturity, num_periods, num_periods_per_year
    )

    price_1 = calculate_bond_price(
        par_value, coupon, yield_to_maturity + blip, num_periods, num_periods_per_year
    )
    price_2 = calculate_bond_price(
        par_value, coupon, yield_to_maturity - blip, num_periods, num_periods_per_year
    )

    price_change = price_1 - price_2
    price_percentage_change = price_change / price
    yield_change = 2 * blip
    modified_duration = - price_percentage_change / yield_change

    return modified_duration

In [ ]:
par_value = 100 
coupon = 3.5
yield_to_maturity = 0.08
periods = 12
num_periods_per_year = 2

convexity = calculate_modified_duration_comp(par_value, coupon, yield_to_maturity, periods, num_periods_per_year, blip=0.0002)
print(f"Modified Duriation: {convexity:.4f}")

Modified Duriation: 4.7807


## Portfolio Duration

## Convexity

Convexity is used to improve the accuracy of the modified duration approximation to the change in the value of the bond. It is important to include it to the approximation for:

* Large changes in the yield-to-maturity
* Bonds with highly non-linear price-yield relationships

Convexity is used because the modified duration is a linear approximation of a non-linear function (i.e. the price).

Convexity can be computed numerically as:

$$
C = 
\frac{1}{2} 
\frac{
    (P_{+dy} + P_{-dy} - 2P) / P
}
{
dy^2
}
$$

In [70]:
def calculate_convexity(
    par_value: float,
    coupon: float,
    yield_to_maturity: float,
    num_periods: int,
    num_periods_per_year: int,
    blip: float = 0.00005,
) -> float:
    price = calculate_bond_price(
        par_value, coupon, yield_to_maturity, num_periods, num_periods_per_year
    )

    price_1 = calculate_bond_price(
        par_value, coupon, yield_to_maturity + blip, num_periods, num_periods_per_year
    )

    price_2 = calculate_bond_price(
        par_value, coupon, yield_to_maturity - blip, num_periods, num_periods_per_year
    )

    convexity = 0.5 * ((price_1 + price_2 - 2 * price) / price) / blip**2
    return convexity

In [71]:
par_value = 100 
coupon = 3.5
yield_to_maturity = 0.08
periods = 12
num_periods_per_year = 2

convexity = calculate_convexity(par_value, coupon, yield_to_maturity, periods, num_periods_per_year, blip=0.0002)
print(f"Convexity: {convexity:.4f}")

Convexity: 13.9753


## Other Risk Measures